In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wildfire-prediction-dataset/valid/wildfire/-73.71631,46.04137.jpg
/kaggle/input/wildfire-prediction-dataset/valid/wildfire/-74.62281,45.80216.jpg
/kaggle/input/wildfire-prediction-dataset/valid/wildfire/-70.19693,53.61931.jpg
/kaggle/input/wildfire-prediction-dataset/valid/wildfire/-71.15229,46.51529.jpg
/kaggle/input/wildfire-prediction-dataset/valid/wildfire/-71.70749,45.72332.jpg
/kaggle/input/wildfire-prediction-dataset/valid/wildfire/-77.82525,48.13819.jpg
/kaggle/input/wildfire-prediction-dataset/valid/wildfire/-68.12459,53.23771.jpg
/kaggle/input/wildfire-prediction-dataset/valid/wildfire/-66.00647,48.16936.jpg
/kaggle/input/wildfire-prediction-dataset/valid/wildfire/-78.47237,46.8756.jpg
/kaggle/input/wildfire-prediction-dataset/valid/wildfire/-75.76655,45.55576.jpg
/kaggle/input/wildfire-prediction-dataset/valid/wildfire/-77.3906,51.6158.jpg
/kaggle/input/wildfire-prediction-dataset/valid/wildfire/-77.20392,46.92026.jpg
/kaggle/input/wildfire-prediction-dataset/v

In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #for data augumentation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout #flatten is mulidimensional to single diemansional layer
import tkinter as tk #to build gui
from tkinter import filedialog #provides a dialog box to select files.
from PIL import Image, ImageTk #ImageTk - Converts images for display in Tkinter GUIs
import numpy as np

In [13]:
# Set up directories
train_dir = r"/kaggle/input/wildfire-prediction-dataset/train"
valid_dir = r"/kaggle/input/wildfire-prediction-dataset/valid"
test_dir = r"/kaggle/input/wildfire-prediction-dataset/test"

# Set up ImageDataGenerators for loading images
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(64, 64), batch_size=32, class_mode='binary')


Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


In [14]:
# Building a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification: wildfire or no wildfire
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [15]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [16]:
# Train the model
history = model.fit(train_generator, validation_data=valid_generator, epochs=2, verbose=1)

Epoch 1/2
946/946 ━━━━━━━━━━━━━━━━━━━━ 126s 131ms/step - accuracy: 0.8774 - loss: 0.2846 - val_accuracy: 0.9376 - val_loss: 0.1654
Epoch 2/2
946/946 ━━━━━━━━━━━━━━━━━━━━ 125s 132ms/step - accuracy: 0.9331 - loss: 0.1780 - val_accuracy: 0.9511 - val_loss: 0.1329


In [17]:
#save the model
model.save("ffd_model.h5")

In [18]:
import tkinter as tk
from tkinter import filedialog  

from PIL import Image, ImageTk  
import numpy as np  
import tensorflow as tf  

from tensorflow.keras.models import load_model

In [21]:
# Function to load and predict an image
def predict_image():
    # Open file dialog to select an image
    file_path = filedialog.askopenfilename()
    if file_path:
        # Display the image in the GUI
        img = Image.open(file_path)
        img = img.resize((200, 200))
        img = ImageTk.PhotoImage(img)    #convert image for tk
        image_label.configure(image=img) #update the image in GUI
        image_label.image = img

        # Preprocess the image for the model
        img_for_model = Image.open(file_path).resize((64, 64))
        img_array = np.array(img_for_model) / 255.0  # Rescale like during training
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Make a prediction
        prediction = model.predict(img_array)[0][0] #extracts the scalar prediction value
        result = "Wildfire" if prediction > 0.5 else "No Wildfire"
        result_label.config(text="Prediction: " + result)

# Setting up the GUI window
root = tk.Tk()
root.title("Forest Fire Detection")
root.geometry("400x400")

# Add widgets
btn = tk.Button(root, text="Upload Image", command=predict_image) #button triggers the predict_image() function when clicked
btn.pack(pady=20)

#Placeholder for displaying the selected image
image_label = tk.Label(root)
image_label.pack()

#Label to display the prediction result
result_label = tk.Label(root, text="Prediction: ", font=("Helvetica", 16))
result_label.pack(pady=20)

#Starts the Tkinter event loop, keeping the GUI active until manually closed
root.mainloop()

TclError: no display name and no $DISPLAY environment variable